In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

/home/suils789/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_csv('sample.csv')

In [3]:
df_ = df.head(10000)

In [4]:
msk = np.random.rand(len(df_)) < 0.8
df_train = df_[msk]

In [5]:
traits_tr = [eval(x) for x in df_train['encodedTraits.indices'].values]

In [6]:
user_indecies = []
trait_indecies = []
for i in range(len(traits_tr)):
    for j in range(len(traits_tr[i])):
        user_indecies.append(df_train.index[i])
        trait_indecies.append(traits_tr[i][j]-1)

In [7]:
trait_values = []
for i in range(len(traits_tr)):
    for j in range(len(traits_tr[i])):
        trait_values.append(1.0)

In [8]:
# set up rank
feature_len = 50
U = tf.Variable(initial_value=tf.truncated_normal([df_.shape[0],feature_len]), name='users')
P = tf.Variable(initial_value=tf.truncated_normal([feature_len,5197]), name='traits')
result = tf.matmul(U, P)
result_flatten = tf.reshape(result, [-1])

In [9]:
# only consider the matrix position where the entry is not null
R = tf.gather(result_flatten, user_indecies * tf.shape(result)[1] + trait_indecies, name='extracting_user_traits')

In [10]:
# cost function
diff_op = tf.subtract(R, trait_values, name='trainig_diff')
diff_op_squared = tf.square(diff_op, name="squared_difference")
base_cost = tf.reduce_sum(diff_op_squared, name="sum_squared_error")

In [11]:
# regularization
lda = tf.constant(.001, name='lambda')
norm_sums = tf.add(tf.reduce_sum(tf.square(U, name='user_abs'), name='user_norm'),
                   tf.reduce_sum(tf.square(P, name='trait_abs'), name='item_norm'))
regularizer = tf.multiply(norm_sums, lda, 'regularizer')

In [12]:
# cost function
loss = base_cost + regularizer
lr = tf.constant(.001, name='learning_rate')
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(lr, global_step, 10000, 0.96, staircase=True)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_step = optimizer.minimize(loss, global_step=global_step)

In [13]:
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

for i in range(1000):
    sess.run(training_step)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [14]:
# calculate accuracy
df_test = df_[~msk]
traits_test = [eval(x) for x in df_test['encodedTraits.indices'].values]
user_indecies_test = []
trait_indecies_test = []
for i in range(len(traits_test)):
    for j in range(len(traits_test[i])):
        user_indecies_test.append(df_test.index[i])
        trait_indecies_test.append(traits_test[i][j]-1)

In [15]:
trait_test_values = []
for i in range(len(traits_test)):
    for j in range(len(traits_test[i])):
        trait_test_values.append(1.0)

In [16]:
# accuracy
R_test = tf.gather(result_flatten, user_indecies_test * tf.shape(result)[1] + trait_indecies_test, name='extracting_user_rate_test')
diff_op_test = tf.subtract(R_test, trait_test_values, name='test_diff')
diff_op_squared_test = tf.abs(diff_op, name="squared_difference_test")

cost_test = tf.div(tf.reduce_sum(tf.square(diff_op_squared_test, name="squared_difference_test"), name="sum_squared_error_test"), df_test.shape[0] * 2, name="average_error")
print (sess.run(cost_test))

nan
